# Quantifying Wildfire Impacts Using the OPERA DIST-HLS Product

### Example below showcases a wildfire event that impacted multiple regions of the greater Los Angeles, California area (01/07/2025-Ongoing).

The Leafmap library provides a suite of tools for interactive mapping and visualization in Jupyter Notebooks Leafmap version 0.30.0 and and later offer tools specifically for accessing NASA Earthdata by building on the newly developed NASA Earthaccess library. Earthaccess provides streamlined access to NASA Earthdata and simplifies the authentication and querying process over previously developed approaches.This notebook is designed to leverage tools within Earthaccess and Leafmap to facility easier access and vizualization of OPERA data products for a user-specified area of interest (AOI). 


## OPERA DIST-ALERT-HLS info
See website https://www.jpl.nasa.gov/go/opera/products/dist-product-suite/

## Import Libraries
Ensure that following dependencies are installed into you environement. This notebook was developed using the `opera_app` environment, which can be found in the [OPERA Applications Github repository](https://github.com/OPERA-Cal-Val/OPERA_Applications).

In [1]:
from datetime import datetime
import numpy as np
import leafmap
import os
import pandas as pd
import rasterio
import shutil

In [ ]:
### Set working directory (modify to your desired path)
working_directory = '/LA_fires'
os.makedirs(working_directory, exist_ok=True)
os.chdir(working_directory)

# Verify the working directory
print("Current working directory:", os.getcwd())

## Authentication 
A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download the data used in this tutorial. You can create an account at the link provided. After establishing an account, the code in the next cell will verify authentication. If this is your first time running the notebook, you will be prompted to enter your Earthdata login credentials, which will be saved in ~/.netrc.

In [3]:
leafmap.nasa_data_login()

## View NASA Earthdata datasets
A tab separated values (TSV) file, made available through the opengeos Github repository, catalogues metadata for more than 9,000 datasets available through NASA Earthdata. In the next cell we load the TSV into a pandas dataframe and view the metadata for the first five (5) Earthdata products

In [4]:
### View Earthdata datasets
earthdata_url = 'https://github.com/opengeos/NASA-Earth-Data/raw/main/nasa_earth_data.tsv'
earthdata_df = pd.read_csv(earthdata_url, sep='\t')
# earthdata_df.head()

## View the available OPERA products
Note above that the `earthdata_df` contains a number of columns with metadata about each available product. the `ShortName` column will be used to produce a new dataframe containing only OPERA products. Let's view the available products and their metadata.

In [5]:
opera_df = earthdata_df[earthdata_df['ShortName'].str.contains('OPERA', case=False)]

## Define an area of interest (AOI) and time period of interest (TOI)
Define an area of interest (AOI) for the wildfire event.

In [11]:
### This cell initializes the AOI and TOI.
AOI = (-120.019, 33.528, -116.219, 34.540) #W, S, E, N; Los Angeles, CA, USA

StartDate_PostFire="2025-1-11T00:00:00"  #Post-fire image start date
EndDate_PostFire="2025-1-14T23:59:59"    #Post-fire image end date

## Query Earthdata and return metadata for OPERA products within the AOI
The `earthaccess` library makes it simple to quickly query NASA's Common Metadata Repository (CMR) and return the associated metadata as a Geodataframe. `Leafmap` has recently added functionality that builds on `earthaccess` to enable interactive viewing of this data. 
In the next cell, the user should specify which OPERA product and the date range of interest. The AOI defined previously is used as the boundary in the query.

### View OPERA Product Shortnames

In [12]:
### Print the available OPERA datasets 
print('Available OPERA datasets:', opera_df['ShortName'].values)

Available OPERA datasets: ['OPERA_L2_CSLC-S1-STATIC_V1' 'OPERA_L2_CSLC-S1_V1'
 'OPERA_L2_RTC-S1-STATIC_V1' 'OPERA_L2_RTC-S1_V1'
 'OPERA_L3_DIST-ALERT-HLS_V1' 'OPERA_L3_DIST-ANN-HLS_V1'
 'OPERA_L3_DSWX-HLS_V1']


### Query the OPERA DIST-ALERT-HLS dataset for the AOI


In [13]:
dist_results_PostFire, dist_gdf_PostFire = leafmap.nasa_data_search(
    short_name='OPERA_L3_DIST-ALERT-HLS_V1',
    cloud_hosted=True,
    bounding_box= AOI,
    temporal=(StartDate_PostFire, EndDate_PostFire),
    count=-1,  # use -1 to return all datasets
    return_gdf=True,
)

Granules found: 8


### See the available DSWx-HLS layers
Functionality within earthaccess enables more more asthetic views of the available layers, as well as displaying the thumbnail. These links are clickable and will download in the browser when clicked. 

In [14]:
dist_results_PostFire[0] #Note this just shows a single MGRS/HLS tile

Collection: {'ShortName': 'OPERA_L3_DIST-ALERT-HLS_V1', 'Version': '1'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -116.19741269, 'Latitude': 34.25029435}, {'Longitude': -115.88645427, 'Latitude': 35.23795494}, {'Longitude': -117.00021983, 'Latitude': 35.24307771}, {'Longitude': -117.00021722, 'Latitude': 34.25292159}, {'Longitude': -116.19741269, 'Latitude': 34.25029435}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2025-01-12T18:44:38.298640Z', 'EndingDateTime': '2025-01-12T18:44:38.298640Z'}}
Size(MB): 0
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-STATUS.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-IND.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-ANOM.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-HIST.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-ANOM-MAX.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-CONF.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-DATE.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-COUNT.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-DUR.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-LAST-DATE.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_GEN-DIST-STATUS.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_GEN-ANOM.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/OPERA_L3_DIST-ALERT-HLS_V1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_GEN-ANOM-MAX.tif', 'https://data.lpdaac.earthda

### View the DIST-ALERT-HLS metadata and footprints

In [15]:
dist_gdf_PostFire.head()

,size,concept-type,concept-id,revision-id,native-id,collection-concept-id,provider-id,format,revision-date,BeginningDateTime,...,ProviderDates,ShortName,RelatedUrls,CloudCover,DayNightFlag,ProductionDateTime,Platforms,URL,Name,geometry
0,0,granule,G3369013897-LPCLOUD,1,OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731...,C2746980408-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-01-14T04:23:10.829Z,2025-01-12T18:44:38.298640Z,...,"[{'Date': '2025-01-14T04:21:04.126105Z', 'Type...",OPERA_L3_DIST-ALERT-HLS_V1,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,1,Day,2025-01-14T04:21:04.126105Z,"[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"POLYGON ((-116.19741 34.25029, -115.88645 35.2..."
1,0,granule,G3369013507-LPCLOUD,1,OPERA_L3_DIST-ALERT-HLS_T11SMU_20250112T183731...,C2746980408-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-01-14T04:22:15.864Z,2025-01-12T18:44:42.413250Z,...,"[{'Date': '2025-01-14T04:19:51.087400Z', 'Type...",OPERA_L3_DIST-ALERT-HLS_V1,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,2,Day,2025-01-14T04:19:51.087400Z,"[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"POLYGON ((-118.08644 34.24811, -116.89400 34.2..."
2,0,granule,G3369253276-LPCLOUD,1,OPERA_L3_DIST-ALERT-HLS_T11SLU_20250112T183731...,C2746980408-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-01-14T10:48:04.412Z,2025-01-12T18:44:45.870830Z,...,"[{'Date': '2025-01-14T10:46:34.548313Z', 'Type...",OPERA_L3_DIST-ALERT-HLS_V1,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,1,Day,2025-01-14T10:46:34.548313Z,"[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"POLYGON ((-119.17150 34.23369, -117.97959 34.2..."
3,0,granule,G3368946588-LPCLOUD,1,OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731...,C2746980408-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-01-14T02:22:56.583Z,2025-01-12T18:44:49.968680Z,...,"[{'Date': '2025-01-14T02:09:17.300882Z', 'Type...",OPERA_L3_DIST-ALERT-HLS_V1,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,2,Day,2025-01-14T02:09:17.300882Z,"[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"POLYGON ((-119.48721 34.22769, -119.06537 34.2..."
4,0,granule,G3369087011-LPCLOUD,1,OPERA_L3_DIST-ALERT-HLS_T11SNT_20250112T183731...,C2746980408-LPCLOUD,LPCLOUD,application/vnd.nasa.cmr.umm+json,2025-01-14T06:44:20.103Z,2025-01-12T18:44:52.934960Z,...,"[{'Date': '2025-01-14T06:41:24.402520Z', 'Type...",OPERA_L3_DIST-ALERT-HLS_V1,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,8,Day,2025-01-14T06:41:24.402520Z,"[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"POLYGON ((-116.47563 33.35025, -116.16983 34.3..."


In [16]:
### Plot the location of the tiles 
dist_gdf_PostFire.explore(fill=False)

In [17]:
dist_gdf_PostFire['geometry'].to_file('DIST-HLS_LA_Fires.geojson',driver='GeoJSON')

## Download data with leafmap
Let's download the data from one of our above queries. In the cell below we specify data from the DIST-ALERT-HLS.

### Create a subdirectory
This will be where the files are downloaded. It will be a subdirectory inside of a directory called `data`, and the directory name will be the date that it was created.

In [18]:
def create_data_directory():
    current_datetime = datetime.now().strftime("%m_%d_%Y")

    # Define the base directory
    base_directory = "data"

    # Create the full path for the new directory
    new_directory_path_PostFire = os.path.join(base_directory, f"data_{current_datetime}")
    # Create the new directory
    os.makedirs(new_directory_path_PostFire, exist_ok=True)

    print(f"Directory '{new_directory_path_PostFire}' created successfully.")

    return new_directory_path_PostFire

directory_path_PostFire = create_data_directory()

Directory 'data/data_01_14_2025' created successfully.


### Download the data
The below will download the data to your newly created subdirectory. Look on your file system for a directory `/data/date/` where `date` is the date the directory was created.

In [19]:
dist_data_PostFire = leafmap.nasa_data_download(dist_results_PostFire, out_dir=directory_path_PostFire)     

 Getting 8 granules, approx download size: 0.0 GB


QUEUEING TASKS | :   0%|          | 0/152 [00:00<?, ?it/s]

File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-STATUS.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-IND.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-ANOM.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-HIST.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-ANOM-MAX.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-CONF.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-DATE.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-COUNT.tif already downloaded
File OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v

PROCESSING TASKS | :   0%|          | 0/152 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/152 [00:00<?, ?it/s]

### Filter to post-event disturbance
We would like to view only disturbance detected after the LA wildfires. To do so, we produce a 'filtered' derivative of the DIST-ALERT-HLS layers based on the date of intitial disturbance detection (from the VEG-DIST-DATE layer). Pixel values in the VEG-DIST-DATE layer correspond the date of initial disturbance detection in days since December 31, 2020. Because the wildfire began 7 January 2025, we filter out disturbance that occurred before this date, corresponding to VEG-DIST-DATE pixel values less than 1467 (the number of since 12/31/2020).

Below we calculate the number of days since December 31, 2020 and store it as a variable, which will be used as input in a subsequent step.

In [20]:
# Define the reference date and the target date
reference_date = datetime(2020, 12, 31)
target_date = datetime(2025, 1, 6)

# Calculate the difference between the two dates
delta = target_date - reference_date

# Get the number of days from the timedelta object
days_since_reference = delta.days

print("Number of days:", days_since_reference)

Number of days: 1467


For this exercise, we are only interested in disturbance detected after the wildfire began on 01/07/2025. Below, we compute a new set of raster layers that are filtered to contain only data following date of interest (day 1467, in this case).

In [21]:
# Initialize a dictionary to store unique IDs and their corresponding file names
layers_dict = {}

# Iterate over files in the directory
for filename in os.listdir(directory_path_PostFire):
    # Check if the filename matches the expected format
    if filename.endswith('.tif'):
        # Split the filename by underscores
        parts = filename.split('_')
        
        # Ensure there are enough parts to avoid index errors
        if len(parts) >= 5:
            # Extract the unique ID (assuming it’s at index 3)
            unique_id = parts[3]  # Adjust the index if needed
            
            # Add the full filename to the dictionary under the corresponding unique ID
            if unique_id not in layers_dict:
                layers_dict[unique_id] = []  # Initialize an empty list if ID not in dictionary
            
            layers_dict[unique_id].append(filename)

# Print the dictionary of unique IDs and their associated file names
print("Dictionary of unique IDs and corresponding files:")
for unique_id, files in layers_dict.items():
    print(f"{unique_id}: {files}")

Dictionary of unique IDs and corresponding files:
T11SKU: ['OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM-MAX.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-HIST.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-ANOM-MAX.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-DIST-COUNT.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-LAST-DATE.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-DIST-DUR.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-DIST-STATUS.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-DIST-DUR.tif', 'OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-DIST-DAT

In [22]:
# Function to generate filtered rasters
def generate_filtered_rasters(layers_dict, threshold, output_subdir='filtered'):
    # Ensure the output subdirectory exists
    output_dir = os.path.join(directory_path_PostFire, output_subdir)
    os.makedirs(output_dir, exist_ok=True)

    # Process each unique_id
    for unique_id, file_list in layers_dict.items():
        # Identify the reference _VEG-DIST-DATE.tif file
        date_file = next((f for f in file_list if f.endswith('_VEG-DIST-DATE.tif')), None)
        if not date_file:
            print(f"Warning: No '_VEG-DIST-DATE.tif' file found for {unique_id}")
            continue

        # Open the _VEG-DIST-DATE.tif file and create a mask based on the threshold
        date_file_path = os.path.join(directory_path_PostFire, date_file)
        with rasterio.open(date_file_path) as src:
            date_data = src.read(1)  # Read the first (and only) band
            date_mask = date_data >= threshold  # Mask where date data exceeds the threshold

        # Apply the mask to each layer file
        for file in file_list:
            # If the file is _VEG-DIST-DATE.tif, apply the threshold and save a filtered version
            if file == date_file:
                with rasterio.open(date_file_path) as src:
                    date_filtered_data = np.where(date_mask, date_data, src.nodata)  # Apply the mask
                    date_filtered_filename = file.replace('.tif', '_filtered.tif')  # Update filename
                    date_filtered_path = os.path.join(output_dir, date_filtered_filename)
                    
                    # Save the filtered _VEG-DIST-DATE.tif raster
                    src_meta = src.meta
                    src_meta.update({"nodata": src.nodata})
                    with rasterio.open(date_filtered_path, 'w', **src_meta) as dest:
                        dest.write(date_filtered_data, 1)  # Write to the first band

                print(f"Generated filtered file: {date_filtered_filename}")
                continue  # Move to the next file in the list

            # If file is _DATA-MASK.tif, copy it directly to the output directory with "_filtered" added
            if file.endswith('_DATA-MASK.tif'):
                data_mask_filtered_filename = file.replace('.tif', '_filtered.tif')
                data_mask_filtered_path = os.path.join(output_dir, data_mask_filtered_filename)
                shutil.copy(os.path.join(directory_path_PostFire, file), data_mask_filtered_path)
                
                print(f"Copied _DATA-MASK file: {data_mask_filtered_filename}")
                continue  # Move to the next file in the list

            else:
                print(f"Processing file: {file}")
                # Open the layer file
                file_path = os.path.join(directory_path_PostFire, file)
                with rasterio.open(file_path) as src:
                    layer_data = src.read(1)  # Read the first band
                    layer_meta = src.meta  # Metadata to use for the output file
                    layer_nodata = src.nodata  # Get the 'nan' value for this layer

                    # Apply the mask: where date_mask is False, set layer_data to layer_nodata
                    filtered_data = np.where(date_mask, layer_data, layer_nodata)

                    # Update the filename to include "_filtered"
                    filtered_filename = file.replace('.tif', '_filtered.tif')
                    filtered_file_path = os.path.join(output_dir, filtered_filename)

                    # Save the filtered raster with the same metadata
                    layer_meta.update({"nodata": layer_nodata})
                    with rasterio.open(filtered_file_path, 'w', **layer_meta) as dest:
                        dest.write(filtered_data, 1)  # Write to the first band
                print(f"Generated filtered file: {filtered_filename}")

# Call the function using the 'days_since_reference' variable as the threshold
generate_filtered_rasters(layers_dict, days_since_reference)


Processing file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM.tif
Generated filtered file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM_filtered.tif
Processing file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM-MAX.tif
Generated filtered file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_GEN-ANOM-MAX_filtered.tif
Processing file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-HIST.tif
Generated filtered file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-HIST_filtered.tif
Processing file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-ANOM-MAX.tif
Generated filtered file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T020917Z_S2A_30_v1_VEG-ANOM-MAX_filtered.tif
Processing file: OPERA_L3_DIST-ALERT-HLS_T11SKU_20250112T183731Z_20250114T02

## Merge filtered rasters
Below we produce a mosaicked version of each layer, which are saved in a new directory called `merged`

In [23]:
import os
import rasterio
from rasterio.merge import merge

def merge_filtered_rasters(filtered_rasters_dir):
    
    # Create a directory for merged rasters
    merged_dir = os.path.join(filtered_rasters_dir, 'merged')
    os.makedirs(merged_dir, exist_ok=True)
    
    # Dictionary to hold lists of filenames for each layer
    layer_dict = {}

    # Iterate through filtered rasters to populate the dictionary
    for filename in os.listdir(filtered_rasters_dir):
        if filename.endswith('_filtered.tif'):
            # Split the filename to extract the layer name
            parts = filename.split('_')
            if len(parts) >= 6:  # Check if there are enough parts to avoid index errors
                layer_name = parts[-2]  # Extract the layer name (second-to-last part)
            if layer_name not in layer_dict:
                layer_dict[layer_name] = []
            layer_dict[layer_name].append(os.path.join(filtered_rasters_dir, filename))

    # Merge rasters for each layer and save them
    for layer_name, files in layer_dict.items():
        # Open the rasters and extract nodata values
        src_files_to_mosaic = [rasterio.open(f) for f in files]
        
        # Get the consistent nodata value for the layer
        nodata_value = src_files_to_mosaic[0].nodata
        
        # Perform the merge with preference for data pixels
        mosaic, out_trans = merge(src_files_to_mosaic, nodata=nodata_value, method='first')
        
        # Create metadata for the merged raster
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "nodata": nodata_value
        })

        # Save the merged raster
        merged_filename = f"{layer_name}_merged.tif"
        merged_filepath = os.path.join(merged_dir, merged_filename)

        with rasterio.open(merged_filepath, 'w', **out_meta) as dest:
            dest.write(mosaic)

        print(f"Merged raster saved as: {merged_filename}")

        # Close all opened raster files
        for src in src_files_to_mosaic:
            src.close()

    return layer_dict

# Call the function with the filtered_rasters directory
filtered_rasters_dir = os.path.join(directory_path_PostFire, 'filtered')
layer_dict = merge_filtered_rasters(filtered_rasters_dir)

Merged raster saved as: GEN-DIST-STATUS_merged.tif
Merged raster saved as: VEG-IND_merged.tif
Merged raster saved as: DATA-MASK_merged.tif
Merged raster saved as: VEG-ANOM-MAX_merged.tif
Merged raster saved as: VEG-DIST-STATUS_merged.tif
Merged raster saved as: VEG-DIST-DATE_merged.tif
Merged raster saved as: VEG-LAST-DATE_merged.tif
Merged raster saved as: GEN-ANOM_merged.tif
Merged raster saved as: VEG-ANOM_merged.tif
Merged raster saved as: GEN-DIST-CONF_merged.tif
Merged raster saved as: VEG-DIST-COUNT_merged.tif
Merged raster saved as: GEN-ANOM-MAX_merged.tif
Merged raster saved as: GEN-LAST-DATE_merged.tif
Merged raster saved as: VEG-DIST-CONF_merged.tif
Merged raster saved as: VEG-HIST_merged.tif
Merged raster saved as: GEN-DIST-DATE_merged.tif
Merged raster saved as: GEN-DIST-COUNT_merged.tif
Merged raster saved as: GEN-DIST-DUR_merged.tif
Merged raster saved as: VEG-DIST-DUR_merged.tif


### Add symbology to tiffs
Some of the DIST-ALERT-HLS layers have embedded symbology. Below we add this symbology to the merged raster layers. If they do not have embedded symbology, they are skipped.

In [24]:
def add_symbology(merged_raster_path, reference_symbology_path):
    try:
        # Read the reference symbology raster
        with rasterio.open(reference_symbology_path) as src:
            # Check if the symbology raster has a colormap
            if 1 in src.colormap(1):
                src_colormap = src.colormap(1)  # Assuming symbology is in band 1
            else:
                print(f"No colormap found for {reference_symbology_path}")
                return  # Exit if no colormap exists

        # Open the merged raster in write mode
        with rasterio.open(merged_raster_path, 'r+') as dst:
            # Write the color map to the first band
            dst.write_colormap(1, src_colormap)

    except Exception as e:
        print(f"Symbology not present for {reference_symbology_path}: {e}...skipping")

symbology_layers = {}

# Loop over each layer in the layer_dict
for layer in layer_dict:
    # Check if we already found a file for this layer
    if layer not in symbology_layers:
        # Loop over each file in the directory
        for filename in os.listdir(directory_path_PostFire):
            # Check if the file is a .tif file
            if filename.endswith('.tif'):
                # Extract the layer name (last part before the extension)
                layer_name = filename.split('_')[-1].split('.')[0]
                
                # Check if the layer name matches the current layer
                if layer_name == layer:
                    # Save the full file path for the first match
                    symbology_layers[layer] = os.path.join(directory_path_PostFire, filename)
                    break  # Stop once the first file for this layer is found

# Print the found file paths for each layer
for layer, path in symbology_layers.items():
    print(f"Layer {layer}: {path}")

# Apply symbology to each merged raster
for layer in layer_dict:
    merged_path = os.path.join(filtered_rasters_dir, 'merged', f"{layer}_merged.tif")
    symbology_path = symbology_layers.get(layer)
    if symbology_path:
        add_symbology(merged_path, symbology_path)
    else:
        print(f"No symbology file found for layer: {layer}")


Layer GEN-DIST-STATUS: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SNT_20250112T183731Z_20250114T064124Z_S2A_30_v1_GEN-DIST-STATUS.tif
Layer VEG-IND: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SMU_20250112T183731Z_20250114T041951Z_S2A_30_v1_VEG-IND.tif
Layer DATA-MASK: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SNT_20250112T183731Z_20250114T064124Z_S2A_30_v1_DATA-MASK.tif
Layer VEG-ANOM-MAX: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-ANOM-MAX.tif
Layer VEG-DIST-STATUS: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SMU_20250112T183731Z_20250114T041951Z_S2A_30_v1_VEG-DIST-STATUS.tif
Layer VEG-DIST-DATE: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SNU_20250112T183731Z_20250114T042104Z_S2A_30_v1_VEG-DIST-DATE.tif
Layer VEG-LAST-DATE: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SLU_20250112T183731Z_20250114T104634Z_S2A_30_v1_VEG-LAST-DATE.tif
Layer GEN-ANOM: data/data_01_14_2025/OPERA_L3_DIST-ALERT-HLS_T11SKU_2

## View the files using Leafmap
We can visualize a few of the layers using the `leafmap` library. After the map renders, click the upper right corner to toggle on/off the layers.

In [25]:
import leafmap

# Initialize a Leafmap map
m = leafmap.Map(center=[40, -100], zoom=4)

# Visualize a few of the layers
layers_viz = ['VEG-DIST-STATUS', 'VEG-ANOM-MAX', 'VEG-DIST-DATE']

# Loop over each layer in layer_dict and add the corresponding merged raster to the map
for layer in layers_viz:
    merged_path = os.path.join(filtered_rasters_dir, 'merged', f"{layer}_merged.tif")
    
    # Check if the merged raster file exists
    if os.path.exists(merged_path):
        # Add the raster to the map with a label
        m.add_raster(merged_path, layer_name=layer, opacity=0.6)

# Display the map
m


Map(center=[34.276311, -118.04433850000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…